# User Analysis

As part of my spotify playlist recommendation project, I will be now engineering user data. I am starting with around 1m rows of (song_id, user_id, num_listens). I also have the "safety scores" of each of these songs, which is an object that looks includes these values, with each being a float from 0-1:

```
threshold_sexual, threshold_harassment, threshold_harassment_threatening,	threshold_hate,	threshold_hate_threatening,	threshold_illicit,	threshold_illicit_violent,	threshold_violence,	threshold_violence_graphic
```

My goal is to determine each user's 85% for each category. This will tell me what they're comfortable listening to, and better inform my safe recommendation system.

In [ ]:
import pandas as pd
import numpy as np

triplets_data = pd.read_csv('triplets_cleaned.csv')

songs_per_user = triplets_data.groupby('user_id').size()

total_users = len(songs_per_user)
mean_songs = songs_per_user.mean()
median_songs = songs_per_user.median()
std_songs = songs_per_user.std()

percentiles = [10, 25, 50, 75, 90, 95, 99]
percentile_values = [songs_per_user.quantile(p/100) for p in percentiles]

print(f"Total users: {total_users:,}")
print(f"Mean songs per user: {mean_songs:.1f}")
print(f"Median songs per user: {median_songs:.1f}")
print(f"Standard deviation: {std_songs:.1f}")
print()

for p, val in zip(percentiles, percentile_values):
    print(f"{p}th percentile: {val:.0f} songs")

print()

cutoff_options = [5, 10, 15, 20, 25, 30]
for cutoff in cutoff_options:
    users_below_cutoff = (songs_per_user < cutoff).sum()
    percentage_below = (users_below_cutoff / total_users) * 100
    print(f"Users with <{cutoff} songs: {users_below_cutoff:,} ({percentage_below:.1f}%)")

recommended_cutoff = max(10, min(30, int(mean_songs * 0.3)))
print(f"\nRecommended cutoff based on data: {recommended_cutoff} songs")
print(f"This captures users with <{(songs_per_user < recommended_cutoff).sum() / total_users * 100:.1f}% of listening data")

Total users: 73,949
Mean songs per user: 13.4
Median songs per user: 8.0
Standard deviation: 17.8

10th percentile: 2 songs
25th percentile: 4 songs
50th percentile: 8 songs
75th percentile: 16 songs
90th percentile: 31 songs
95th percentile: 45 songs
99th percentile: 88 songs

Users with <5 songs: 22,241 (30.1%)
Users with <10 songs: 43,141 (58.3%)
Users with <15 songs: 53,562 (72.4%)
Users with <20 songs: 59,634 (80.6%)
Users with <25 songs: 63,479 (85.8%)
Users with <30 songs: 66,041 (89.3%)

Recommended cutoff based on data: 10 songs
This captures users with <58.3% of listening data


Create Dataframe

In [14]:
import pandas as pd
import numpy as np

triplets_data = pd.read_csv('triplets_cleaned.csv')
all_user_ids = triplets_data['user_id'].unique()

safety_category_names = ['sexual', 'harassment', 'harassment_threatening', 'hate',
                        'hate_threatening', 'illicit', 'illicit_violent', 'violence', 'violence_graphic']

threshold_column_names = [f'threshold_{category}' for category in safety_category_names]

initial_dataframe = pd.DataFrame({
    'user_id': all_user_ids,
    'song_count': np.nan,
    'threshold_method': '',
    **{col: np.nan for col in threshold_column_names}
})

initial_dataframe.to_csv('user_data.csv', index=False)

# Calculate Threshold of Each Category
Calculating the weighted 85% percentile threshold for each of the respective categories. This algorithm may seem complicated, but it's just:


* **Pair and sort**
  Places songs in increasing safety score order, carrying along each song's listen-count weight.
* **Compute cumulative weight**
  It running-totals those listen counts so you know, at each score, what fraction of all listens falls at or below that score.
* **Determine the 85% cutoff**
  It multiplies user's total listen-count by 0.85 to find the “target weight” where 85% of plays have lower or equal safety scores.
* **Locate the breakpoint**
  It finds the first index in the cumulative weights that meets or exceeds that target weight.
* **Return the score**
  The safety score at that index is your weighted 85th percentile.


In [2]:
import pandas as pd
import json
import numpy as np
from scipy import stats

def calculate_weighted_percentile(score_values, listen_count_weights, target_percentile):
    # Sort safety scores from lowest to highest
    sorted_indices = np.argsort(score_values)
    sorted_scores = score_values[sorted_indices]
    sorted_weights = listen_count_weights[sorted_indices]

    # Calculate cumulative weights, weighting the vategory value by how many listens the song has
    cumulative_weights = np.cumsum(sorted_weights)
    total_weight = cumulative_weights[-1]

    # Find the target weight position (e.g., 85% of total weight)
    target_weight = target_percentile / 100.0 * total_weight
    insertion_index = np.searchsorted(cumulative_weights, target_weight)

    return sorted_scores[insertion_index]

In [3]:
existing_thresholds_df = pd.read_csv('user_data.csv')
triplets_listening_data = pd.read_csv('triplets_cleaned.csv')
songs_with_safety_scores = pd.read_csv('joined_songs_cleaned.csv')
songs_with_safety_scores['parsed_safety_scores'] = songs_with_safety_scores['safety_scores'].apply(json.loads)
safety_category_list = ['sexual', 'harassment', 'harassment/threatening', 'hate',
                       'hate/threatening', 'illicit', 'illicit/violent', 'violence', 'violence/graphic']

for safety_category in safety_category_list:
    column_name = f'safety_{safety_category.replace("/", "_")}'
    songs_with_safety_scores[column_name] = songs_with_safety_scores['parsed_safety_scores'].apply(lambda x: x[safety_category])

merged_user_song_data = triplets_listening_data.merge(songs_with_safety_scores, on='song_id', how='inner')

minimum_songs_for_personalization = 5
user_percentile_threshold = 85

conservative_default_thresholds = {}
for safety_category in safety_category_list:
    safety_column_name = f'safety_{safety_category.replace("/", "_")}'
    conservative_default_thresholds[safety_category] = np.percentile(songs_with_safety_scores[safety_column_name], 95)

users_needing_processing = existing_thresholds_df[existing_thresholds_df['song_count'].isna()]

for user_index, user_row in users_needing_processing.iterrows():
    current_user_id = user_row['user_id']
    current_user_songs = merged_user_song_data[merged_user_song_data['user_id'] == current_user_id]

    user_song_count = len(current_user_songs)
    existing_thresholds_df.loc[user_index, 'song_count'] = user_song_count

    if user_song_count < minimum_songs_for_personalization:
        for safety_category in safety_category_list:
            threshold_column = f'threshold_{safety_category.replace("/", "_")}'
            existing_thresholds_df.loc[user_index, threshold_column] = conservative_default_thresholds[safety_category]
        existing_thresholds_df.loc[user_index, 'threshold_method'] = 'conservative_default'
    else:
        for safety_category in safety_category_list:
            safety_column_name = f'safety_{safety_category.replace("/", "_")}'
            user_safety_scores = current_user_songs[safety_column_name].values
            user_listen_counts = current_user_songs['listen_count'].values

            personalized_threshold = calculate_weighted_percentile(user_safety_scores, user_listen_counts, user_percentile_threshold)
            threshold_column = f'threshold_{safety_category.replace("/", "_")}'
            existing_thresholds_df.loc[user_index, threshold_column] = personalized_threshold

        existing_thresholds_df.loc[user_index, 'threshold_method'] = f'weighted_{user_percentile_threshold}th_percentile'
    if user_index % 1000 == 0:
        existing_thresholds_df.to_csv('user_data.csv', index=False)
        print(f"Processed {user_index} users")


/tmp/ipython-input-3-2950821678.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  existing_thresholds_df = pd.read_csv('user_data.csv')


Processed 48000 users
Processed 49000 users
Processed 50000 users
Processed 51000 users
Processed 52000 users
Processed 53000 users
Processed 54000 users
Processed 55000 users
Processed 56000 users
Processed 57000 users
Processed 58000 users
Processed 59000 users
Processed 60000 users
Processed 61000 users
Processed 62000 users
Processed 63000 users
Processed 64000 users
Processed 65000 users
Processed 66000 users
Processed 67000 users
Processed 68000 users
Processed 69000 users
Processed 70000 users
Processed 71000 users
Processed 72000 users
Processed 73000 users


# Calculate New Thresholds
We previously gave users with less than 5 songs, song-based conservative thresholds. This is to prevent problematic cold starting.

Now that we have user data, lets give users user-based thresholds, for a more realistic interpretation of thresholds.

We will just find the 85th percentile of user's listening thresholds per category, across all users that have 5 or more songs. This isn't weighted, like before.

In [6]:
import pandas as pd
import numpy as np

user_data = pd.read_csv('user_data.csv')

# Remove users with blank or 0 song listens, forgot to do before
initial_count = len(user_data)
user_data = user_data.dropna(subset=['song_count'])
user_data = user_data[user_data['song_count'] > 0]
removed_count = initial_count - len(user_data)

safety_categories = ['sexual', 'harassment', 'harassment_threatening', 'hate',
                    'hate_threatening', 'illicit', 'illicit_violent', 'violence', 'violence_graphic']

personalized_users = user_data[user_data['threshold_method'] == 'weighted_85th_percentile']

new_conservative_defaults = {}
for category in safety_categories:
    col = f'threshold_{category}'
    new_conservative_defaults[col] = np.percentile(personalized_users[col], 85)

conservative_mask = user_data['threshold_method'] == 'conservative_default'
for category in safety_categories:
    col = f'threshold_{category}'
    user_data.loc[conservative_mask, col] = new_conservative_defaults[col]

user_data.to_csv('user_data_updated.csv', index=False)

print(f"Removed {removed_count} users with blank or 0 song listens")
print(f"Updated {conservative_mask.sum()} users with new conservative defaults")
print(f"Total users in final dataset: {len(user_data)}")
print("\nNew conservative thresholds (85th percentile of user thresholds):")
for category in safety_categories:
    print(f"{category}: {new_conservative_defaults[f'threshold_{category}']:.6f}")

Removed 10893 users with blank or 0 song listens
Updated 19050 users with new conservative defaults
Total users in final dataset: 63056

New conservative thresholds (85th percentile of user thresholds):
sexual: 0.204482
harassment: 0.197983
harassment_threatening: 0.019397
hate: 0.008030
hate_threatening: 0.003347
illicit: 0.006253
illicit_violent: 0.000095
violence: 0.403610
violence_graphic: 0.016081


In [5]:
import pandas as pd
import json

songs = pd.read_csv('joined_songs_cleaned.csv')
songs['parsed_scores'] = songs['safety_scores'].apply(json.loads)

thresholds = {
    'sexual': 0.204482,
    'harassment': 0.197983,
    'harassment/threatening': 0.019397,
    'hate': 0.008030,
    'hate/threatening': 0.003347,
    'illicit': 0.006253,
    'illicit/violent': 0.000095,
    'violence': 0.403610,
    'violence/graphic': 0.016081
}

print("Songs exceeding thresholds:")
for category, threshold in thresholds.items():
    exceeds = sum(songs['parsed_scores'].apply(lambda x: x[category] > threshold))
    pct = (exceeds / len(songs)) * 100
    print(f"{category}: {exceeds} songs ({pct:.1f}%)")

print("\nThreshold percentiles in song distribution:")
for category, threshold in thresholds.items():
    scores = songs['parsed_scores'].apply(lambda x: x[category])
    percentile = (scores <= threshold).mean() * 100
    print(f"{category}: {percentile:.1f}th percentile")

Songs exceeding thresholds:
sexual: 289 songs (6.0%)
harassment: 362 songs (7.5%)
harassment/threatening: 379 songs (7.9%)
hate: 343 songs (7.1%)
hate/threatening: 352 songs (7.3%)
illicit: 373 songs (7.7%)
illicit/violent: 381 songs (7.9%)
violence: 362 songs (7.5%)
violence/graphic: 322 songs (6.7%)

Threshold percentiles in song distribution:
sexual: 94.0th percentile
harassment: 92.5th percentile
harassment/threatening: 92.1th percentile
hate: 92.9th percentile
hate/threatening: 92.7th percentile
illicit: 92.3th percentile
illicit/violent: 92.1th percentile
violence: 92.5th percentile
violence/graphic: 93.3th percentile


# Get Average Number of Plays Per Song, Per User

In [1]:
import pandas as pd
import numpy as np

print("Loading triplets data...")
triplets_data = pd.read_csv('triplets_cleaned.csv')

print("Calculating average plays per song for each user...")
user_avg_plays = triplets_data.groupby('user_id')['listen_count'].mean().reset_index()
user_avg_plays.columns = ['user_id', 'avg_plays_per_song']

print("Loading existing user data...")
user_data = pd.read_csv('user_data_updated.csv')
initial_columns = user_data.columns.tolist()

print("Merging average plays data...")
user_data = user_data.merge(user_avg_plays, on='user_id', how='left')

user_data.to_csv('user_data_updated.csv', index=False)

Loading triplets data...
Calculating average plays per song for each user...
Loading existing user data...
Merging average plays data...
